#### deskriptive Statistik interaktiv

Unterschiedlichen Verteilungsfunktionen können interaktiv parametriert und in einem Diagramm dargestellt werden. Zu der dargestellten Funktion können zudem weitere Kennwerte dargestellt werden.

**Auswählbare Funktionen:**
- Normalverteilung (Gauß-Funktion)
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.norm.html#scipy.stats.norm    

- Gamma-Poisson-Verteilung (negative Binomialverteilung)
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.nbinom.html#scipy.stats.nbinom

- Poisson-Verteilung https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.poisson.html#scipy.stats.poisson

- Weibull-Verteilung https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.weibull_min.html


In [ ]:
# resourcen
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson, norm, nbinom, weibull_min, mode

#set backend for interactive toolbar
%matplotlib nbagg

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#interaktive Schalter
# Signalauswahl
distribFuns = ['Poissonverteilung', 'Gamma-Poisson-Verteilung','Normalverteilung','Weibull-Verteilung']
distrib_select = widgets.Dropdown(description='', options=distribFuns, value='Normalverteilung',layout= widgets.Layout(width='75%',height='auto'))

# Funktionsparameter
#
#
layout= widgets.Layout(width='50%',height='auto')
style = {'description_width': 'initial'}
size = widgets.BoundedIntText(value=1000, min=20, max=100000,\
                              description='Datensatzgröße:',\
                             layout= widgets.Layout(width='75%',height='auto'))
#
# Gauss: µ: Erwartungswert, \sigma: Standardabweichung 
mu = widgets.FloatSlider(min=-150, max=150, value=0, description="µ:",\
                         tooltips="Gauss: Erwartungswert. Poisson: Erwartungswert und Varianz",\
                        continuous_update=False, layout= widgets.Layout(width='75%',height='auto'))
sigma = widgets.FloatSlider(min=0.1, max=5, value=1, description='$\sigma$:', tooltips="Standardabweichung",\
                            layout= widgets.Layout(width='75%',height='auto'))
# Weibull : - Skalen- und Formparameter
lambda_wbl = widgets.FloatSlider(min=0, max=1, value=0.1, step=0.01, description='$\lambda$:', tooltips="Skalenparameter $\lambda$.%1 \over \lambda > 0$.",\
                                layout= widgets.Layout(width='75%',height='auto'))
form_wbl = widgets.FloatSlider(min=0, max=15, value=1, description="k:", tooltips="Formparamter k. k > 0",layout= widgets.Layout(width='75%',height='auto'))
# Gamma-Poisson => negative Binomialverteilung: n: Anzahl Erfolge, p: Einzelwahrscheinlichkeit 
n = widgets.IntSlider(min=0, max=500, value = 10, description='n:', tooltips="Anzahl der Erfolge n. n > 0", layout= widgets.Layout(width='75%',height='auto'))
p = widgets.FloatSlider(min=0, max=1, value = 0.1, description='p:', tooltips="Wahrscheinlichkeit eines einzelnen Erfolgs. $0 \leq p \leq 1$",\
                       layout= widgets.Layout(width='75%',height='auto'))
# Poisson: Erwartungswert und Varianz µ (geteilt mit Gauss - Erwartungswert)
#
# Parameter der Verteilungsfunktionen
# Quantil - Auswahl
quantile = widgets.FloatSlider(min= 0, max=100, value=50, description="Quantil-Auswahl /%",layout= widgets.Layout(width='50%',height='auto'))
#

# Visualisierungsoptionen
toggle_1 = widgets.ToggleButtons(options=['pdf/pmf', 'cdf'],\
                                 description='',\
                                 disabled=False,
                                 button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                 tooltips=['zeige Dichte- bzw. Wahrscheinlichkkeitsfunktion', 'zeige Verteilungsfunktion'],
                                )
toggle_2 = widgets.ToggleButtons(options=['line','boxplot', 'violinplot'],
                                 description='',
                                 disabled=False,
                                 button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                 tooltips=['line-plot','Boxer-und-Whisker Ansicht', 'Violin Ansicht'],
                                )
#
# plot limits
x_range = widgets.IntRangeSlider(min= -200, max=200, value=[-5,5],description='$x_{range}$', layout= widgets.Layout(width='50%',height='auto'))
#
BtnReset = widgets.Button(description="reset", button_style='warning')

In [ ]:
def draw_rvs(dict_distribution, ax):
# Datensatz der Verteilungsfunktion darstellen
    x = dict_distribution['x']
    y = dict_distribution['rvs']    
    ax.clear()
    ax.set_title("{}".format(distrib_select.value))
#     ax.hist(y, normed=True, histtype='stepfilled', alpha=0.2)
    ax.hist(y)
    ax.set_xlim(x_range.value[0], x_range.value[-1])
#     ax.set_xlim(x[0], x[-1])

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.yaxis.grid(True, linestyle='-', which='major',\
                  color='lightgrey',alpha=0.5)

In [ ]:
def draw_params(dict_distribution, ax):
#     x = np.linspace(x_range.value[0],x_range.value[-1],size.value)
    x = dict_distribution['x']
    distrib_f = dict_distribution['distrib_f']    
    median = dict_distribution['median']
    mean = dict_distribution['mean']
    var = dict_distribution['var']
    std = dict_distribution['std']
    rms = dict_distribution['rms']
    
    if toggle_1.value == 'pdf/pmf':
        mode = dict_distribution['mode_cdf']
        y = dict_distribution['prob_f']  
        while (True):
            #
            if distrib_select.value == 'Poissonverteilung':
                mode_y = poisson.pmf(mode, mu.value)
                mean_y = poisson.pmf(mean, mu.value)
                median_y = poisson.pmf(median, mu.value)
                break
            #
            if distrib_select.value == 'Gamma-Poisson-Verteilung':
                mode_y = distrib_f.pmf(mode)
                mean_y = distrib_f.pmf(mean)
                median_y = distrib_f.pmf(median)
                break
            #
            if distrib_f != 'NaN':
                mode_y = distrib_f.pdf(mode)
                mean_y = distrib_f.pdf(mean)
                median_y = distrib_f.pdf(median)
                break
            #
    else:
        mode = dict_distribution['mode_cdf']
        y = dict_distribution['cdf']
        if distrib_select.value == 'Poissonverteilung':
            mode_y = poisson.cdf(mode, mu.value)
            mean_y = poisson.cdf(mean, mu.value)
            median_y = poisson.cdf(median, mu.value)
        if distrib_f != 'NaN':
            mode_y = distrib_f.cdf(mode)
            mean_y = distrib_f.cdf(mean)
            median_y = distrib_f.cdf(median)

    # mean: vertikale Linie, gestrichelt, grau, KOmmentar oben mit Pfeil, Text: µ (wert)
    # varianz: standardabweichung: - horzontale linie für gültigen Bereich
    # rms
    # modus: gauß: 0
    # median: gauß: mean
    ax.clear()
    ax.set_title("Parameter zur {}".format(distrib_select.value))
    #Verteilung nochmal zeichnen und dann die entsprechenden Informationen eintragen
    ax.plot(x, y, color='C1', lw=2)
    
    # vertikale Line bis zum Funktionswert zeichnen
    # angaben für axhline bzw. axvline in %!
    yLim = ax.get_ylim()
    yH = mean_y/yLim[-1] 
    #
    #Mittelwert
    ax.axvline(x=mean, ymin=0, ymax=yH, ls='--',color='grey',alpha=0.5)
    ax.annotate(r"$\bar{{x}}_{{{}}}$".format(mean),
                 xy=(mean, mean_y), xycoords='data',
                 xytext=(-50,30), textcoords='offset points',
                 bbox=dict(boxstyle="round", fc="0.6"),
                 arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=90,rad=10"))
    #
    # mode
    yH = mode_y/yLim[-1]    
    #
    ax.axvline(x=mode, ymin=0, ymax=yH) 
    ax.annotate("mode",
                 xy=(mode, mode_y), xycoords='data',
                 xytext=(-50,30), textcoords='offset points',
                 bbox=dict(boxstyle="round", fc="0.6"),
                 arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=90,rad=10"))
    #
    # median
    yH = median_y/yLim[-1]
    #
    ax.axvline(x=median, ymin=0, ymax=yH)
    ax.annotate("median",
                 xy=(median, median_y), xycoords='data',
                 xytext=(-50,30), textcoords='offset points',
                 bbox=dict(boxstyle="round", fc="0.6"),
                 arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=90,rad=10"))
    #
    # Normalverteilung:
    # 1-sigma, 2-sigma und 3-sigma kennzeichnen
    if (distrib_select.value == 'Normalverteilung' and toggle_1.value == 'pdf/pmf'):
        #
#         
#         x_l = distrib_f.ppf(1-0.6827)
#         x_h = distrib_f.ppf(0.6827)
#         y_h = distrib_f.pdf(x_l)
#         ax.plot((x_l,x_h),(y_h,y_h),'k-')
        
        m_1_sigma = distrib_f.ppf(1-0.6827)
        p_1_sigma = distrib_f.ppf(0.6827)
        m_2_sigma = distrib_f.ppf(1-0.9545)
        p_2_sigma = distrib_f.ppf(0.9545)
        m_3_sigma = distrib_f.ppf(1-0.9973)
        p_3_sigma = distrib_f.ppf(0.9973)
        #
        yH_s1n = distrib_f.pdf( m_1_sigma)
        yH_s1p = distrib_f.pdf( p_1_sigma)
        yH_s2n = distrib_f.pdf( m_2_sigma)
        yH_s2p = distrib_f.pdf( p_2_sigma)
        yH_s3n = distrib_f.pdf( m_3_sigma)
        yH_s3p = distrib_f.pdf( p_3_sigma)
        #
        ax.axvline(x=m_1_sigma, ymin=0, ymax=yH_s1n/yLim[-1])
        ax.axvline(x=p_1_sigma, ymin=0, ymax=yH_s1p/yLim[-1])
        ax.axvline(x=m_2_sigma, ymin=0, ymax=yH_s2n/yLim[-1])
        ax.axvline(x=p_2_sigma, ymin=0, ymax=yH_s2p/yLim[-1])
        ax.axvline(x=m_3_sigma, ymin=0, ymax=yH_s3n/yLim[-1])
        ax.axvline(x=p_3_sigma, ymin=0, ymax=yH_s3p/yLim[-1])
        
        # bereich markieren +/- sigma
        ax.fill_between(x, 0, y, where =(y >= (yH_s3n)),\
                   facecolor='blue', alpha=0.3)
        ax.fill_between(x,0,y, where =(y >= (yH_s2n)),\
                   facecolor='blue', alpha=0.2)
        ax.fill_between(x,0,y, where =(y >= (yH_s1n)),\
                   facecolor='blue', alpha=0.1)
        
        # # hozizontale Linie für +/- sigma
#         ax.annotate('', xy=(m_1_sigma, yH_s1n*yLim[-1]), xycoords='data',
#                   xytext=(p_1_sigma, yH_s1p*yLim[-1]), textcoords='data',
#                   arrowprops=dict(arrowstyle="<->", connectionstyle="bar",
#                                   ec="k", shrinkA=5, shrinkB=5))
        ax.plot((m_1_sigma,p_1_sigma),(yH_s1n, yH_s1n), ls='-', color='green')
        ax.annotate(r'$\pm\sigma$', xy=(mu.value, yH_s1n), xycoords='data')
        #
#         ax.annotate('', xy=(m_2_sigma, yH_s2n*yLim[-1]), xycoords='data',
#                      xytext=(p_2_sigma, yH_s2p*yLim[-1]), textcoords='data',
#                      arrowprops=dict(arrowstyle="<->", connectionstyle="bar",
#                                      ec="k",shrinkA=5, shrinkB=5))
        ax.plot((m_2_sigma,p_2_sigma), (yH_s2n, yH_s2n),ls='-', color='green')
        ax.annotate(r'$\pm2\sigma$', xy=(mu.value, yH_s2n), xycoords='data')
        #
#         ax.annotate('', xy=(m_3_sigma, yH_s3n*yLim[-1]), xycoords='data',
#                     xytext=(p_3_sigma, yH_s3p*yLim[-1]), textcoords='data',
#                     arrowprops=dict(arrowstyle="<->", connectionstyle="bar"))
        ax.plot((m_3_sigma,p_3_sigma), (yH_s3n, yH_s3n),ls='-', color='green')
        ax.annotate(r'$\pm3\sigma$', xy=(mu.value, yH_s3n), xycoords='data')
   
                     
#     ax.set_xlim(x[0], x[-1]) 
    ax.set_xlim(x_range.value[0], x_range.value[-1])
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)


In [ ]:
def draw_custom(dict_distribution, ax):
    
    x = dict_distribution['x']
    distrib_f = dict_distribution['distrib_f']    
    median = dict_distribution['median']
    mean = dict_distribution['mean']
    var = dict_distribution['var']
    std = dict_distribution['std']
    rms = dict_distribution['rms']
    #
    if toggle_1.value == 'pdf/pmf':
        mode = dict_distribution['mode_cdf']
        y = dict_distribution['prob_f'] 
        title_str = "Dichtefunktion der {}".format(distrib_select.value)
        while (True):
            #
            if distrib_select.value == 'Poissonverteilung':
                xQ1 = poisson.ppf(quantile.value/100, mu.value)
                yQ1 = poisson.pmf(xQ1, mu.value)
                mode_y = poisson.pmf(mode, mu.value)
                mean_y = poisson.pmf(mean, mu.value)
                median_y = poisson.pmf(median, mu.value)
                break
            #
            if distrib_select.value == 'Gamma-Poisson-Verteilung':
                xQ1 = distrib_f.ppf(quantile.value/100)
                yQ1 = distrib_f.pmf(xQ1)
                mode_y = distrib_f.pmf(mode)
                mean_y = distrib_f.pmf(mean)
                median_y = distrib_f.pmf(median)
                break
            #
            if distrib_f != 'NaN':
                xQ1 = distrib_f.ppf(quantile.value/100)
                yQ1 = distrib_f.pdf(xQ1)
                mode_y = distrib_f.pdf(mode)
                mean_y = distrib_f.pdf(mean)
                median_y = distrib_f.pdf(median)
                break
            #
    else:
        mode = dict_distribution['mode_cdf']
        y = dict_distribution['cdf']
        title_str = "Verteilungsfunktion der {}".format(distrib_select.value)
        #
        #
        if distrib_select.value == 'Poissonverteilung':
            xQ1 = poisson.ppf(quantile.value/100, mu.value)
            yQ1 = poisson.cdf(xQ1,mu.value)
            mode_y = poisson.cdf(mode, mu.value)
            mean_y = poisson.cdf(mean, mu.value)
            median_y = poisson.cdf(median, mu.value)
        #
        if distrib_f != 'NaN':
            xQ1 = distrib_f.ppf(quantile.value/100)
            yQ1 = distrib_f.cdf(xQ1)
            mode_y = distrib_f.cdf(mode)
            mean_y = distrib_f.cdf(mean)
            median_y = distrib_f.cdf(median)
        #
    ax.clear()
        
    ax.set_title(title_str)
    ax.plot(x, y, ls='-',lw=2)
    ax.set_xlim(x_range.value[0], x_range.value[-1])   

    yLim = ax.get_ylim()
    yq = np.max(y)*0.5
    #
    # info für quantil 
    txt_q1 = '\n'.join((
        r'$\mathrm{%.2f Prozent}$' % (quantile.value, ),
        r'$\mathrm{%.2f Wahrscheinlichkeit}$' % (yQ1, )
        ))
    # 
    # quantile
    ax.axvline(x=xQ1, ymin=0, ymax=0.5, ls='--', color='darkgrey')
    ax.annotate(txt_q1,
                 xy=(xQ1, yq), xycoords='data',
                 xytext=(-2,20), textcoords='offset points',
                 bbox=dict(boxstyle="round", fc="0.6"),
                 arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=90,rad=10"))
    
    if toggle_2.value == 'boxplot':
        ax.clear()
        ax.boxplot(y)
        ax.set_xticklabels([])
    
    if toggle_2.value == 'violinplot':
        ax.clear()
        ax.violinplot(y, showmeans=True, showmedians=True, showextrema=True,\
                           quantiles=[0.685])
        ax.set_xticklabels([])
      
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)
    
    # parameter in text box 
    textstr = '\n'.join((
    r'$%s$' % (distrib_select.value, ),
    r'$\mu=%.2f$' % (mu.value, ),
    r'$\mathrm{median}=%.2f$' % (median_y, ),
    r'$\mathrm{modus}=%.2f$' % (mode_y, ),
    r'$\mathrm{mean}=%.2f$' % (mean_y, ),
    r'$\sigma=%.2f$' % (sigma.value, ),
    r'$\sigma^2=%.2f$' % (var, ),
    r'$\mathrm{rms}=%.2f$' % (rms, )))
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # place a text box in upper left in axes coords
    if distrib_select.value == 'Normalverteilung':
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,\
                verticalalignment='top', bbox=props)
    else:
        ax.text(0.75, 0.95, textstr, transform=ax.transAxes, fontsize=10,\
                verticalalignment='top', bbox=props)

In [ ]:
def reset_controls(*btn):
    # Reset Button:
    distrib_select.value = 'Normalverteilung'
    size.value = 1000
    mu.value = 0
    sigma.value = 0
    lambda_wbl.value = 0
    form_wbl.value = 0
    n.value = 0
    p.value = 0
    quantile.value = 50
    toggle_1.value = 'pdf/pmf'
    toggle_2.value = 'line'
    x_range.value = [-5,5]
    update_view()

In [ ]:
# interaktive Visualisierung:
#
#- Figuren Zeichnen:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8,8))

#
# 1. create a callback which updates the plot when a control-value has changed
def update_view(*args):

    # selections
    distribType = distrib_select.value
        
    # Zeitbereich aktualisieren
#     x_view = np.linspace(x_range.value[0],x_range.value[-1],size.value)
    #-----------------------------------------------------------
    # Datensatz nach der gewählten Verteilungsfunktion erzeugen
    #   
    # datensatz mit beliebiger größe: rvs({...}) Random variates
    # abszisse: ppf({...}}) Percent point function (inverse of cdf — percentiles).
    # Dichtefunktion/Wahrscheinlichkeitsfunktion: pdf({...}})/pmf({...}})
    # kummulierte Dichtefunktion: cdf({...}}) Cumulative density function.
    # Parameter sind teils in der Klasse der Verteilung enthalten:
    # - Mittelwert, -Median, -Standardabweichung, -Varianz,
    # mode: mode(a[, axis]) Returns an array of the modal (most common) value in the passed array.
    # mode https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mode.html#scipy.stats.mode
    # rms: sqrt(Mittelwert)
    #
    if (distribType == "Normalverteilung"):
        mu.disabled = False
        mu.min = -50
        sigma.disabled = False
        # Diagram-Bereich umschalten damit Funktion sichtbar
        x_range.min = mu.value -10
        x_range.max =  mu.value +10
        nv = norm(mu.value, sigma.value)
        xnv = np.arange(norm.ppf(0.001,mu.value, sigma.value),norm.ppf(0.999,mu.value, sigma.value), 1/size.value)
        cdf = nv.cdf(xnv)
        pdf = nv.pdf(xnv)
        distributionSelected = {\
                 'x': xnv,\
                 'distrib_f': nv,\
                 'rvs': nv.rvs(size=size.value),\
                 'prob_f': pdf,\
                 'cdf': cdf,\
                 'mean': nv.mean(),\
                 'median': nv.median(),\
                 'mode_cdf': (mode(cdf).mode),\
                 'mode_pdf': (mode(pdf).mode),\
                 'std': nv.std(),\
                 'var': nv.var(),\
                 'rms': np.sqrt(nv.mean())\
                }
        #
#         Steuerelemente ausblenden
        lambda_wbl.disabled = True
        form_wbl.disabled = True
        n.disabled = True
        p.disabled = True
 
        
            
    if (distribType == "Poissonverteilung"):
        mu.disabled = False
        mu.min = 0
        # Diagram-Bereich umschalten damit Funktion sichtbar
        x_range.min = 0
        x_range.max = mu.value +20
        xp = np.linspace(poisson.ppf(0.001, mu.value), poisson.ppf(0.999,mu.value), size.value)
        distributionSelected = {\
                 'x': xp,\
                 'distrib_f': 'NaN',\
                 'rvs': poisson.rvs(mu.value, size = size.value),\
                 'prob_f': poisson.pmf(xp, mu.value),\
                 'cdf': poisson.cdf(xp, mu.value),\
                 'mean': poisson.mean(mu.value),\
                 'median': poisson.median(mu.value),\
                 'mode_cdf': (mode(poisson.cdf(xp, mu.value)).mode),\
                 'mode_pdf': (mode(poisson.pmf(xp, mu.value)).mode),\
                 'std': poisson.std(mu.value),\
                 'var': poisson.var(mu.value),\
                 'rms': np.sqrt(poisson.mean(mu.value))\
                }
        # steuerelemente ausblenden
        sigma.disabled = True
        lambda_wbl.disabled = True
        form_wbl.disabled = True
        n.disabled = True
        p.disabled = True
                            
    if (distribType == "Gamma-Poisson-Verteilung"):
        n.disabled = False
        p.disabled = False
        # Diagram-Bereich umschalten damit Funktion sichtbar
        x_range.min = 0
        x_range.max = n.value +20
        xnb = np.arange(nbinom.ppf(0.001,n.value,p.value), nbinom.ppf(0.999,n.value,p.value), 1/size.value)
        nBinom = nbinom(n.value, p.value)
        pmf = nBinom.pmf(xnb)
        cdf = nBinom.cdf(xnb)
        distributionSelected = {\
                 'x': xnb,\
                 'distrib_f': nBinom,\
                 'rvs': nBinom.rvs(size=size.value),\
                 'prob_f': pmf,\
                 'cdf': cdf,\
                 'mean': nBinom.mean(),\
                 'median': nBinom.median(),\
                 'mode_cdf': (mode(cdf).mode),\
                 'mode_pdf': (mode(pmf).mode),\
                 'std': nBinom.std(),\
                 'var': nBinom.var(),\
                 'rms': np.sqrt(nBinom.mean())\
                }
        # steuerelemente ausblenden
        lambda_wbl.disabled = True
        form_wbl.disabled = True
        mu.disabled = True
        sigma.disabled = True

        
    if (distribType == "Weibull-Verteilung"):
        lambda_wbl.disabled = False
        form_wbl.disabled = False
        # Diagram-Bereich umschalten damit Funktion sichtbar
        x_range.min = 0
        x_range.max = 10
        xwbl = np.arange(weibull_min.ppf(0.001,form_wbl.value), weibull_min.ppf(0.999,form_wbl.value),1/size.value)
        wbl = weibull_min(form_wbl.value)
        pdf = wbl.pdf(xwbl)
        cdf = wbl.cdf(xwbl)
        distributionSelected = {\
                 'x': xwbl,\
                 'distrib_f': wbl,\
                 'rvs': wbl.rvs(size=size.value),\
                 'prob_f': pdf,\
                 'cdf': cdf,\
                 'mean': wbl.mean(),\
                 'median': wbl.median(),\
                 'mode_cdf': (mode(cdf).mode),\
                 'mode_pdf': (mode(pdf).mode),\
                 'std': wbl.std(),\
                 'var': wbl.var(),\
                 'rms': np.sqrt(wbl.mean())\
                }
        mu.disabled = True
        sigma.disabled = True
        n.disabled = True
        p.disabled = True
    #
    # diagramma aktualisieren
    draw_rvs(distributionSelected, axes[0])
    draw_params(distributionSelected, axes[1])
    draw_custom(distributionSelected, axes[2])
    
#     distributionSelected.clear()
#     fig.tight_layout()
    
    
#--------------------------------------------------------
# 2. die Callback-Funktion mit der Funktion 'observe' den Steuerelementen zuweisen
    
distrib_select.observe(update_view, 'value')
size.observe(update_view, 'value')
mu.observe(update_view, 'value')
sigma.observe(update_view, 'value')
lambda_wbl.observe(update_view, 'value')
form_wbl.observe(update_view, 'value')
n.observe(update_view, 'value')
p.observe(update_view, 'value')
quantile.observe(update_view, 'value')
toggle_1.observe(update_view, 'value')
toggle_2.observe(update_view, 'value')
x_range.observe(update_view, 'value')
BtnReset.on_click(reset_controls)

#--------------------------------------------------------
# Anwendung starten
#
# Diagramme einmal zeichnen
update_view()
#
# mit 'widgets.VBox / .HBox' die Steuerelemente arangieren
box_layout = widgets.Layout(display='inline-flex',flex_flow='row',align_items='flex-start',width='90%')

widgets.VBox([widgets.HBox([widgets.VBox([distrib_select,size]),\
             widgets.VBox([mu,sigma,lambda_wbl,form_wbl,n,p]),\
             widgets.VBox([toggle_1,toggle_2])],\
                           layout= box_layout),
              quantile, x_range,BtnReset])